Step 0: On the top right, next to the RAM graph, select change execution environment to GPU T4.

In [ ]:
!git clone https://github.com/hkchengrex/XMem.git
!pip install -r XMem/requirements.txt
!sh XMem/scripts/download_models.sh

Cloning into 'XMem'...
remote: Enumerating objects: 616, done.
remote: Counting objects: 100% (353/353), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 616 (delta 260), reused 215 (delta 215), pack-reused 263 (from 2)
Receiving objects: 100% (616/616), 267.58 KiB | 9.91 MiB/s, done.
Resolving deltas: 100% (362/362), done.
  Cloning https://github.com/cheind/py-thin-plate-spline to /tmp/pip-req-build-_zeyw92m
  Running command git clone --filter=blob:none --quiet https://github.com/cheind/py-thin-plate-spline /tmp/pip-req-build-_zeyw92m
  Resolved https://github.com/cheind/py-thin-plate-spline to commit f6995795397118b7d0ac01aecd3f39ffbfad9dee
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
!git clone https://github.com/jorgegalinmor/swift-project.git
!pip install -r swift-project/requirements.txt

Cloning into 'swift-project'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 66 (delta 34), reused 60 (delta 31), pack-reused 0 (from 0)
Receiving objects: 100% (66/66), 246.70 KiB | 5.03 MiB/s, done.
Resolving deltas: 100% (34/34), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 119.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 113.9 MB/s eta 0:00:00
   ━━

Each step will have one cell defining variables that will need to be modified each time and another that will do the analysis needed to complete the steps using the variables defined in the first one.  
Only modify the first cell.  
Make sure to run both cells.  
Step 0 (No need to do each time, it will erase your event detection configuration): Copy config file to drive.

In [ ]:
config_file_path = "drive/MyDrive/swift_project_config.ini"
!cp swift-project/config.ini {config_file_path}

Step 1: Upload video file to the side bar on the directory view. Initiate variables pointing to the video and label folders.


In [ ]:
#Set variables to the proper values. the video should be inside the videos/ folder:
from pathlib import Path

config_file_path = Path("drive/MyDrive/swift_project_config.ini")
video_path = Path("drive/MyDrive/videos/Camera2/Video107/20230708_062650_tp00107.mp4")
per_frames = 10
original_fps = 30
result_fps = original_fps / per_frames
class_label_folder = video_path.parent / "SegmentationClass"
class_label_folder.parent.mkdir(parents=True, exist_ok=True)
object_label_folder = video_path.parent / "SegmentationObject"
object_label_folder.parent.mkdir(parents=True, exist_ok=True)

Step 2: Extract frames.

In [ ]:
!video-toimg --per {per_frames} {video_path}

{'duration': None,
 'in_files': ['drive/MyDrive/videos/Camera2/Video107/20230708_062650_tp00107.mp4'],
 'per': 10,
 'start': 0}
100% 25410/25410 [03:36<00:00, 117.60it/s]


Step 3: Download the first frame, and possibly more every, for example 1000 frames.  
Step 4: Go to app.cvat.ai, create a new task on the swift project (it should have the same labels as prior videos), and upload these frames. Open the task and label the frames using the magic wand on the left, selecting the Segment Anything 2.0 model.  
Export the dataset labels as Segmentation Mask 1.1.  
Upload the SegmentationClass and SegmentationObject folder contents to the similarly named folders in the video folder. Add labelmap.txt to the video folder as well.  
Step 5. Use XMem to track the labels:

In [ ]:
import os

video_path = Path(video_path)

frames_dir_path = video_path.parent / Path(video_path.stem)

objects_input = Path("drive/MyDrive/videos/XMem-input/Objects/")
classes_input = Path("drive/MyDrive/videos/XMem-input/Classes/")

class_labels_dir = video_path.parent / "SegmentationClass/"
object_labels_dir = video_path.parent / "SegmentationObject/"

objects_input_images = objects_input / "JPEGImages" / Path('_'.join(video_path.parent.parts[1:]))
objects_input_images.parent.mkdir(parents=True, exist_ok=True)
objects_input_labels = objects_input / "Annotations" / Path('_'.join(video_path.parent.parts[1:]))
objects_input_labels.parent.mkdir(parents=True, exist_ok=True)
classes_input_images = classes_input / "JPEGImages" / Path('_'.join(video_path.parent.parts[1:]))
classes_input_images.parent.mkdir(parents=True, exist_ok=True)
classes_input_labels = classes_input / "Annotations" / Path('_'.join(video_path.parent.parts[1:]))
classes_input_labels.parent.mkdir(parents=True, exist_ok=True)

if objects_input_images.exists() or objects_input_images.is_symlink():
    objects_input_images.unlink()
objects_input_images.symlink_to(os.path.relpath(frames_dir_path, start=objects_input_images.parent), target_is_directory=True)
if objects_input_labels.exists() or objects_input_labels.is_symlink():
    objects_input_labels.unlink()
objects_input_labels.symlink_to(os.path.relpath(object_labels_dir, start=objects_input_labels.parent), target_is_directory=True)

if classes_input_images.exists() or classes_input_images.is_symlink():
    classes_input_images.unlink()
classes_input_images.symlink_to(os.path.relpath(frames_dir_path, start=classes_input_images.parent), target_is_directory=True)
if classes_input_labels.exists() or classes_input_labels.is_symlink():
    classes_input_labels.unlink()
classes_input_labels.symlink_to(os.path.relpath(class_labels_dir, start=classes_input_labels.parent), target_is_directory=True)

output_path = Path("drive/MyDrive/videos/XMem-output") / Path('_'.join(video_path.parent.parts[1:]))

objects_output = output_path / "Objects/"
objects_output.mkdir(parents=True, exist_ok=True)

classes_output = output_path / "Classes/"
classes_output.mkdir(parents=True, exist_ok=True)

In [ ]:
!python3 XMem/eval.py --output {objects_output} --dataset G --generic_path {objects_input}

save_all is forced to be true in generic evaluation mode.
Hyperparameters read from the model weights: C^k=64, C^v=512, C^h=64
Single object mode: False
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100% 97.8M/97.8M [00:00<00:00, 325MB/s]
Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100% 44.7M/44.7M [00:00<00:00, 194MB/s]
  0% (0 of 1) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--/content/XMem/eval.py:167: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=not args.benchmark):
100% (1 of 1) |##########################| Elapsed Time: 0:32:41 Time:  0:32:41
Total processing time: 1120.2146951293932
Total processed frames: 2541
FPS: 2.268315181945096
Max allocated memory (MB): 3983.196289

In [ ]:
!python3 XMem/eval.py --output {classes_output} --dataset G --generic_path {classes_input}

save_all is forced to be true in generic evaluation mode.
Hyperparameters read from the model weights: C^k=64, C^v=512, C^h=64
Single object mode: False
  0% (0 of 1) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--/content/XMem/eval.py:167: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=not args.benchmark):
100% (1 of 1) |##########################| Elapsed Time: 0:19:36 Time:  0:19:36
Total processing time: 902.3828669433584
Total processed frames: 2541
FPS: 2.81587793062509
Max allocated memory (MB): 3282.7041015625


Optional Compress and download:labels for the whole video:

In [ ]:
from google.colab import files
!zip -r ./XMem-output.zip {output_path}
files.download("./XMem-output.zip")

Se han truncado las últimas 5000 líneas del flujo de salida.
  adding: drive/MyDrive/videos/XMem-output/MyDrive_videos_Camera2_Video107/Objects/MyDrive_videos_Camera2_Video107/00016250.png (deflated 27%)
  adding: drive/MyDrive/videos/XMem-output/MyDrive_videos_Camera2_Video107/Objects/MyDrive_videos_Camera2_Video107/00016280.png (deflated 28%)
  adding: drive/MyDrive/videos/XMem-output/MyDrive_videos_Camera2_Video107/Objects/MyDrive_videos_Camera2_Video107/00016270.png (deflated 27%)
  adding: drive/MyDrive/videos/XMem-output/MyDrive_videos_Camera2_Video107/Objects/MyDrive_videos_Camera2_Video107/00016290.png (deflated 28%)
  adding: drive/MyDrive/videos/XMem-output/MyDrive_videos_Camera2_Video107/Objects/MyDrive_videos_Camera2_Video107/00016300.png (deflated 28%)
  adding: drive/MyDrive/videos/XMem-output/MyDrive_videos_Camera2_Video107/Objects/MyDrive_videos_Camera2_Video107/00016310.png (deflated 28%)
  adding: drive/MyDrive/videos/XMem-output/MyDrive_videos_Camera2_Video107/Object

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Step 6: Check the configuration in swifts-project/config.ini, and input the correct camera parameters. Everything else can remain as it is for now. Use mask-processing to detect events, with hugging-face's VideoMAE model to perform classification.

In [ ]:
!cd swift-project/ && git pull

remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 3), reused 2 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 1.11 KiB | 567.00 KiB/s, done.
From https://github.com/jorgegalinmor/swift-project
   3b23f27..ab74f86  main       -> origin/main
Updating 3b23f27..ab74f86
Fast-forward
 video_event_callback.py | 8 +-------
 1 file changed, 1 insertion(+), 7 deletions(-)


In [ ]:
!cp {video_path.parent}/labelmap.txt {objects_input}/JPEGImages/labelmap.txt

cp: cannot stat '{video_path.parent}/labelmap.txt': No such file or directory


In [ ]:
model = "jorgegalinmor/swift-events-classification-model"
mask_output = Path("drive/MyDrive/videos/MaskProcessingOutput") / Path('_'.join(video_path.parent.parts[1:]))
mask_output.mkdir(parents=True, exist_ok=True)
!python3 swift-project/mask_processing.py {objects_input_images} {output_path} {mask_output} --model_path  {model} --fps {int(result_fps)} --config {config_file_path}

Se han truncado las últimas 5000 líneas del flujo de salida.
Distance: 0.012963, rolling_avg: 0.019734672899878326, bound: 0.300000, triggered False
Calculating distance between object 4, of type: Nest, 2, and object 0, of type: Nest, 2
Distance: 0.141836, rolling_avg: 0.1414878314242322, bound: 0.300000, triggered False
Calculating distance between object 4, of type: Nest, 2, and object 3, of type: Nest, 2
Distance: 0.012963, rolling_avg: 0.019734672899878326, bound: 0.300000, triggered False
Reading drive/MyDrive/videos/XMem-output/MyDrive_videos_Camera2_Video107/Objects/MyDrive_videos_Camera2_Video107/00017300.png
Reading drive/MyDrive/videos/XMem-output/MyDrive_videos_Camera2_Video107/Classes/MyDrive_videos_Camera2_Video107/00017300.png
Calculating speed for object 0, of type: Nest, 2
Internal speed: 0.014700, external speed: 0.001003, total: 0.015703, bound: 0.700000
Calculating speed for object 3, of type: Nest, 2
Internal speed: 0.118500, external speed: 0.056991, total: 0.17549

Step 7: Compress and download the contents of the MaskProcessingOutput, since the google colab files are temporary only.

In [ ]:
from google.colab import files
!zip -r MaskProcessingOutput.zip {mask_output}
files.download("MaskProcessingOutput.zip")

Se han truncado las últimas 5000 líneas del flujo de salida.
  adding: drive/MyDrive/videos/MaskProcessingOutput/MyDrive_videos_Camera2_Video107/MyDrive_videos_Camera2_Video107/ClosenessCallbackOutput/Snapshots/closeness_callback_00007470_00000000_parental_feeding_snapshots/00007480.jpg (deflated 0%)
  adding: drive/MyDrive/videos/MaskProcessingOutput/MyDrive_videos_Camera2_Video107/MyDrive_videos_Camera2_Video107/ClosenessCallbackOutput/Snapshots/closeness_callback_00007470_00000000_parental_feeding_snapshots/00007490.jpg (deflated 0%)
  adding: drive/MyDrive/videos/MaskProcessingOutput/MyDrive_videos_Camera2_Video107/MyDrive_videos_Camera2_Video107/ClosenessCallbackOutput/Snapshots/closeness_callback_00007470_00000000_parental_feeding_snapshots/00007500.jpg (deflated 0%)
  adding: drive/MyDrive/videos/MaskProcessingOutput/MyDrive_videos_Camera2_Video107/MyDrive_videos_Camera2_Video107/ClosenessCallbackOutput/Snapshots/closeness_callback_00007470_00000000_parental_feeding_snapshots/00

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Step 8: If the results are not satisfactory, you can modify the event thresholds and lengths in swifts_project/config.ini and repeat the steps from step 6 (or if the sessions and temporary files are lost, repeat the steps from the beggining, editing config.ini before step 6).

Step 9 (Optional): Remove the results from google drive after downloading them to save space there:

In [ ]:
!rm -r  {mask_output}
!rm -r {output_path}
!rm -r {frames_dir_path}
!rm -r {objects_input}
!rm -r {classes_input}

rm: cannot remove '{mask_output}': No such file or directory
rm: cannot remove '{output_path}': No such file or directory
rm: cannot remove '{frames_dir_path}': No such file or directory
rm: cannot remove '{objects_input}': No such file or directory
rm: cannot remove '{classes_input}': No such file or directory


Step 10 (Optional): Remove input data to save google drive space:

In [ ]:
!rm -r {class_labels_dir}
!rm -r {object_labels_dir}
!rm {video_path}